In [2]:
import dendropy as dp
import numpy as np
import re, sys, os
import pandas as pd

In [3]:
target_type = 'type1'
tree = dp.Tree.get_from_path('set_test_template_r0_nMig12_migAge1.1_splitAge1.33_prunedLowSamp.nexus.tree', 'nexus')

In [4]:
for i in tree.leaf_node_iter():
    i.taxon.label = re.sub('^_|^ |_$| $', '', i.taxon.label)

def check_monophyly(node, target_type):
    tip_types = []
    tips = []
    for i in node.leaf_iter():
        tips.append(i)
        tip_types.append(re.split('_| ', i.taxon.label)[1])
    return( [all([j == target_type for j in tip_types]), tips] )

monophyletic_nodes = []
all_imports = []
visited_tips = []
visited_nodes = []
for tip in tree.leaf_node_iter():
    if tip in visited_tips:
        continue
    if(re.split('_| ', tip.taxon.label)[1] == target_type):
        monophyletic_ancestors = []
        for ancestor in tip.ancestor_iter():
            if ancestor in visited_nodes:
                continue
            is_monophyletic = check_monophyly(ancestor, target_type) 
            if is_monophyletic[0]:
                monophyletic_ancestors.append([ancestor, ancestor.distance_from_root()])
                for child_tip in is_monophyletic[1]:
                    visited_tips.append([child_tip, child_tip.distance_from_root()])
                visited_nodes.append(ancestor)
            else:
                all_imports.append([ancestor, ancestor.distance_from_root() - ancestor.edge_length])# Because we want to count singletons too!
                visited_tips.append([tip, tip.distance_from_root()])
                visited_nodes.append(ancestor)
                break
        if len(monophyletic_ancestors) > 0:
            oldest_monophyletic_ancestor = monophyletic_ancestors[-1] 
            if not (oldest_monophyletic_ancestor in monophyletic_nodes):
                monophyletic_nodes.append(oldest_monophyletic_ancestor)
        

In [5]:
visited_tips = []
importation_nodes = []
for tip in tree.leaf_node_iter():
    if(len(re.findall(target_type, tip.taxon.label)) > 0):
        if(tip in visited_tips):
            #print('I am alredy a visited tip')
            continue
        else:
            print('I am in tip'+tip.taxon.label)
            print('I have not been here before')
        #print('I have visited '+str(len(visited_tips))+' tips')
        ancestors_of_tip = []
        for ancestor in tip.ancestor_iter():
            is_monophyletic = check_monophyly(ancestor, target_type=target_type)
            ancestors_of_tip.append(ancestor)
            if(not is_monophyletic[0]):
                most_recent_decendants = check_monophyly(ancestors_of_tip[len(ancestors_of_tip)-2], target_type=target_type)[1]
                [visited_tips.append(i) for i in most_recent_decendants]
                importation_nodes.append(ancestor)
                break
        #print('Now I have visited '+str(len(visited_tips)))
        print('I have '+str(len(ancestors_of_tip))+' monophyletic ancestors')

I am in tip11_type1_1.53
I have not been here before
I have 3 monophyletic ancestors
I am in tip14_type1_1.44
I have not been here before
I have 2 monophyletic ancestors
I am in tip56_type1_1.8
I have not been here before
I have 2 monophyletic ancestors
I am in tip64_type1_1.73
I have not been here before
I have 1 monophyletic ancestors
I am in tip96_type1_1.59
I have not been here before
I have 3 monophyletic ancestors


In [6]:
importation_nodes


[<Node object at 0x7f53cae37550: 'None' (None)>,
 <Node object at 0x7f53cae37d60: 'None' (None)>,
 <Node object at 0x7f53cae46f40: 'None' (None)>,
 <Node object at 0x7f53cae4da60: 'None' (None)>,
 <Node object at 0x7f53cae4d580: 'None' (None)>]

In [7]:
import_ages = [i.distance_from_root() for i in importation_nodes] # From here find the first importation
np.min(import_ages)

0.6632636919336057

In [8]:
0.51688+0.14639

0.66327

In [13]:
tip_ages = [i.distance_from_root() for i in visited_tips]
np.max(tip_ages)

1.798978273472032

In [91]:
age_last_sample = re.split('_', visited_tips[tip_ages.idxmax()][0].taxon.label)[2]
node_ages = [i.distance_from_root() for i in tree.postorder_internal_node_iter()]

In [97]:
node_ages = [i.distance_from_root() for i in tree.]

1.7188529432708306

In [95]:
np.max(node_ages)

1.825924016268837

In [7]:
# Get all tip ages
tip_ages = []
for i in tree.leaf_node_iter():
    tip_ages.append([i.taxon.label, i.annotations.get_value('location'), i.distance_from_root()])

tip_ages = pd.DataFrame(tip_ages)
tip_ages.columns = ['tip_label', 'type', 'date']
tip_ages.head()

,tip_label,type,date
0,1_type0_1.35,0,1.351354
1,2_type0_1.38,0,1.376085
2,3_type0_1.3,0,1.299793
3,4_type0_1.28,0,1.283897
4,5_type0_1.37,0,1.365684


In [8]:

# Find age of first human case
location_human = 'type1' #this would be in the args
first_human_case = tip_ages.date[tip_ages.type == location_human].min()
first_human_case

nan

In [14]:


# Sample tips with high sampling probabilty after first human case
print('Sampling non human samples with probability '+str(non_human_sampling_prop_high)+' after first human case')
tips_to_remove = []
for i in range(tip_ages.shape[0]):
    if tip_ages.date[i] < first_human_case:
        tips_to_remove.append(tip_ages.tip_label[i])
        print('Prunning (opportunistic)'+tip_ages.tip_label[i])
    elif tip_ages.type[i] != location_human and tip_ages.date[i] >= first_human_case:
        if np.random.binomial(1, non_human_sampling_prop_high) == 0:
            tips_to_remove.append(tip_ages.tip_label[i])
            print('Prunning (opportunistic)'+tip_ages.tip_label[i])
            
tree_opportunistic_sampling = tree.clone(depth = 1)
tree_opportunistic_sampling.prune_taxa_with_labels(tips_to_remove)
tree_opportunistic_sampling.write_to_path(re.sub('.nexus.tree', '_prunedOppSamp.nexus.tree', tree_file_name), 'nexus')

aln_opportunistic_sampling = alignment.clone(depth = 1)
prune_alignment(aln_opportunistic_sampling, tips_to_remove).write_to_path(re.sub('.fasta', '_prunedOppSamp.fasta', aln_file_name), 'fasta')



NameError: name 'non_human_sampling_prop_high' is not defined

In [16]:
import os, sys, re, subprocess, sqlite3

replicates = [i for i in range(5)] # number of replicates to run

template = 'test_template.xml' # Template xml
prefix = 'set_test_template_r' # the prefix we want to use
beast_path = '~/phyloApps/beast2.6.3/bin/beast'
seqgen_path = '~/phyloApps/Seq-Gen-1.3.4/source/seq-gen' # Maybe include clock rate and genome size as arguments here?
iqtree_path = '~/phyloApps/iqtree-2.1.3-Linux/bin/iqtree2' 
lsd_path = '~/phyloApps/lsd-0.3beta/bin/lsd_unix'

for i in replicates[:1]:
    replicate_xml = open(template, 'r').read()
    output_name = prefix+str(i)
    replicate_xml = re.sub('OUTPUT_FILE_NAME', output_name, replicate_xml)
    open(output_name + '.xml', 'w').writelines(replicate_xml)
    os.system(beast_path + ' ' +output_name+'.xml') # simulate trees
    tree_processing = re.split(' |\n', os.popen('./treePostProcessing.py -t ' + output_name + '.nexus.tree -lh 1 -rc 1e-4 -seqGen ' + seqgen_path).read()) # manipulate trees

    sim_tree_file = tree_processing[-4]
    sim_aln_file = tree_processing[-2]
       
    subsampling_command = './subsample_trees.py -t ' + sim_tree_file + ' -s ' + sim_aln_file + ' -lh '+ '1' + ' -ph ' + '0.8' + ' -pl ' + '0.05' 
    
    subsampled_output = re.split(' |\n', os.popen(subsampling_command).read())

    opp_sampling_aln = subsampled_output[-2]
    opp_sampling_tree = subsampled_output[-3]

    low_samp_aln = subsampled_output[-4]
    low_samp_tree = subsampled_output[-5]

    high_samp_aln = subsampled_output[-6]
    high_samp_tree = subsampled_output[-7]

    estimated_trees = []
    iqtree_command = iqtree_path + ' -s SEQ_ALN_NAME -m GTR+G'
    lsd_command = lsd_path + ' -i SEQ_ALN_NAME -d DATE_ALN_NAME -r a -c'
    for aln in [sim_aln_file, opp_sampling_aln, low_samp_aln, high_samp_aln]:
        os.system(re.sub('SEQ_ALN_NAME', aln, iqtree_command)) # Estimate tree
        os.system('./make_lsd_dates.py -s '+aln)
        os.system(re.sub('DATE_ALN_NAME', re.sub('.fasta', '.date', aln), re.sub('SEQ_ALN_NAME', aln+'.treefile', lsd_command))) # Run lsd
        # capture lsd output and store in variable
        estimated_trees.append(aln+'.treefile.result.date.nexus')

# The estimated trees file names are stored in estimated_trees, and the other ones in variables above

    sim_tree_stats = os.popen('./find_monophyletic.py -t '+sim_tree_file+' -f nexus -l type1').readlines()
    sim_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(sim_tree_stats) if i[0] in [1, 3, 5, 7]]
    


/home/sebastiand/phyloApps/beast2.6.3/bin/beast: line 57: java: command not found
Sequence Generator - seq-gen
Version 1.3.4
(c) Copyright, 1996-2017 Andrew Rambaut and Nick Grassly
Institute of Evolutionary Biology, University of Edinburgh

Originally developed at:
Department of Zoology, University of Oxford

Random number generator seed: -5302675776961924052

Simulations of 123 taxa, 29903 nucleotides
  for 1 tree(s) with 1 dataset(s) per tree

Branch lengths of trees multiplied by 0.0001

Rate homogeneity of sites.
Model = HKY: Hasegawa, Kishino & Yano (1985)
  transition/transversion ratio = 2 (K=4)
  with nucleotide frequencies equal.
Time taken: 0.154629 seconds
Checkpoint (set_test_template_r0_nMig21_migAge0.54_splitAge1.78.fasta.ckp.gz) indicates that a previous run successfully finished
Use `-redo` option if you really want to redo the analysis and overwrite all output files.
Use `--redo-tree` option if you want to restore ModelFinder and only redo tree search.
Use `--undo` op

Reading the tree 1
Estimating the root position on all branches using fast method ...
Pre-estimating the position of the root without using temporal constraints ...
Optimizing the root position on the branch 1 ... 0.0001046854
Optimizing the root position on the branch 2 ... 0.0001046854
Optimizing the root position on the branch 3 ... 0.0001034358
Optimizing the root position on the branch 4 ... 0.0001046648
Optimizing the root position on the branch 5 ... 0.0001030449
Optimizing the root position on the branch 6 ... 0.0001020419
Optimizing the root position on the branch 7 ... 0.0001023215
Optimizing the root position on the branch 8 ... 0.0001032090
Optimizing the root position on the branch 9 ... 0.0001016455
Optimizing the root position on the branch 10 ... 0.0000984742
Optimizing the root position on the branch 11 ... 0.0001021756
Optimizing the root position on the branch 12 ... 0.0001031231
Optimizing the root position on the branch 13 ... 0.0001035047
Optimizing the root posit

Checkpoint (set_test_template_r0_nMig21_migAge0.54_splitAge1.78_prunedOppSamp.fasta.ckp.gz) indicates that a previous run successfully finished
Use `-redo` option if you really want to redo the analysis and overwrite all output files.
Use `--redo-tree` option if you want to restore ModelFinder and only redo tree search.
Use `--undo` option if you want to continue previous run when changing/adding options.


Optimizing the root position on the branch 1 ... 0.0000763874
Optimizing the root position on the branch 2 ... 0.0000763874
Optimizing the root position on the branch 3 ... 0.0000753121
Optimizing the root position on the branch 4 ... 0.0000753121
Optimizing the root position on the branch 5 ... 0.0000738271
Optimizing the root position on the branch 6 ... 0.0000732219
Optimizing the root position on the branch 7 ... 0.0000748587
Optimizing the root position on the branch 8 ... 0.0000778340
Optimizing the root position on the branch 9 ... 0.0000772828
Optimizing the root position on the branch 10 ... 0.0000775935
Optimizing the root position on the branch 11 ... 0.0000765337
Optimizing the root position on the branch 12 ... 0.0000768426
Optimizing the root position on the branch 13 ... 0.0000760773
Optimizing the root position on the branch 14 ... 0.0000760773
Optimizing the root position on the branch 15 ... 0.0000837675
Optimizing the root position on the branch 16 ... 0.0000860183
O

Checkpoint (set_test_template_r0_nMig21_migAge0.54_splitAge1.78_prunedLowSamp.fasta.ckp.gz) indicates that a previous run successfully finished
Use `-redo` option if you really want to redo the analysis and overwrite all output files.
Use `--redo-tree` option if you want to restore ModelFinder and only redo tree search.
Use `--undo` option if you want to continue previous run when changing/adding options.
Checkpoint (set_test_template_r0_nMig21_migAge0.54_splitAge1.78_prunedHighSamp.fasta.ckp.gz) indicates that a previous run successfully finished
Use `-redo` option if you really want to redo the analysis and overwrite all output files.
Use `--redo-tree` option if you want to restore ModelFinder and only redo tree search.
Use `--undo` option if you want to continue previous run when changing/adding options.


Reading the tree 1
Estimating the root position on all branches using fast method ...
Pre-estimating the position of the root without using temporal constraints ...
Optimizing the root position on the branch 1 ... 0.0000784046
Optimizing the root position on the branch 2 ... 0.0000784046
Optimizing the root position on the branch 3 ... 0.0000743450
Optimizing the root position on the branch 4 ... 0.0000762571
Optimizing the root position on the branch 5 ... 0.0000819942
Optimizing the root position on the branch 6 ... 0.0000769081
Optimizing the root position on the branch 7 ... 0.0000764299
Optimizing the root position on the branch 8 ... 0.0000750242
Optimizing the root position on the branch 9 ... 0.0000768627
Optimizing the root position on the branch 10 ... 0.0000802455
Optimizing the root position on the branch 11 ... 0.0000777899
Optimizing the root position on the branch 12 ... 0.0000823886
Optimizing the root position on the branch 13 ... 0.0000823886
Optimizing the root posit

OperationalError: unrecognized token: ".54_splitAge1"

In [54]:
    con = sqlite3.connect("hostjumps.db")

    #create database cursor to fetch **TABLE NAME**
    cur = con.cursor() 

    #create database table to store **TABLE NAME**, called "**TABLE NAME**"
    #columns include X, Y, Z
    table_name = re.sub('_nMig.+', '', sim_tree_file)
    table_name
    cur.execute('CREATE TABLE '+table_name+'(tree_name, tmrca, jumps, first_migration_age, first_detected_case)')
    con.commit()


In [55]:
tmrca = re.sub('[.]nexus.+', '', re.sub('.+splitAge', '', sim_tree_file))
num_migrations = re.sub('_.+', '', re.sub('.+_nMig', '', sim_tree_file))
first_migration_age = re.sub('_.+', '', re.sub('.+migAge', '', sim_tree_file))
first_detected_case = '\'NA\''
summary_stats = tmrca+','+num_migrations+','+first_migration_age+','+first_detected_case
print(summary_stats)

cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'simulated_tree\','+summary_stats+')')
con.commit()

1.78,21,0.54,'NA'


In [74]:
non_painted_tree = re.sub('nexus', 'newick', sim_tree_file)
sim_non_painted_tree_stats = os.popen('./find_monophyletic.py -t '+non_painted_tree+' -f newick -l '+target_type).readlines()
sim_non_painted_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(sim_non_painted_tree_stats) if i[0] in [1, 3, 5, 7]]
sim_non_painted_tree_stats = ','.join(sim_non_painted_tree_stats)
sim_non_painted_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'simulated_non_painted_tree\','+sim_non_painted_tree_stats+')')
con.commit()

'1.892353098084817,17,0.24448182678965097,0.28612702168785153'

In [78]:
high_sampling_tree = re.sub('[.]nexus.+', '_prunedHighSamp.nexus.tree', sim_tree_file)
high_sampling_tree_stats = os.popen('./find_monophyletic.py -t '+high_sampling_tree+' -f nexus -l '+target_type).readlines()
high_sampling_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(high_sampling_tree_stats) if i[0] in [1, 3, 5, 7]]
high_sampling_tree_stats = ','.join(high_sampling_tree_stats)
high_sampling_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'simulated_high_sampling_tree\','+high_sampling_tree_stats+')')
con.commit()


In [79]:
low_sampling_tree = re.sub('[.]nexus.+', '_prunedLowSamp.nexus.tree', sim_tree_file)
low_sampling_tree_stats = os.popen('./find_monophyletic.py -t '+low_sampling_tree+' -f nexus -l '+target_type).readlines()
low_sampling_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(low_sampling_tree_stats) if i[0] in [1, 3, 5, 7]]
low_sampling_tree_stats = ','.join(low_sampling_tree_stats)
low_sampling_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'simulated_low_sampling_tree\','+low_sampling_tree_stats+')')
con.commit()

In [80]:
opp_sampling_tree = re.sub('[.]nexus.+', '_prunedOppSamp.nexus.tree', sim_tree_file)
opp_sampling_tree_stats = os.popen('./find_monophyletic.py -t '+opp_sampling_tree+' -f nexus -l '+target_type).readlines()
opp_sampling_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(opp_sampling_tree_stats) if i[0] in [1, 3, 5, 7]]
opp_sampling_tree_stats = ','.join(opp_sampling_tree_stats)
opp_sampling_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'simulated_opp_sampling_tree\','+opp_sampling_tree_stats+')')
con.commit()

In [81]:
estimated_trees

['set_test_template_r0_nMig21_migAge0.54_splitAge1.78.fasta.treefile.result.date.nexus',
 'set_test_template_r0_nMig21_migAge0.54_splitAge1.78_prunedOppSamp.fasta.treefile.result.date.nexus',
 'set_test_template_r0_nMig21_migAge0.54_splitAge1.78_prunedLowSamp.fasta.treefile.result.date.nexus',
 'set_test_template_r0_nMig21_migAge0.54_splitAge1.78_prunedHighSamp.fasta.treefile.result.date.nexus']

In [88]:
estimated_complete_tree = estimated_trees[0]
estimated_complete_tree_stats = os.popen('./find_monophyletic.py -t '+estimated_complete_tree+' -f nexus -l '+target_type).readlines()
estimated_complete_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(estimated_complete_tree_stats) if i[0] in [1, 3, 5, 7]]
estimated_complete_tree_stats = ','.join(estimated_complete_tree_stats)
estimated_complete_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'estimated_complete_tree\','+estimated_complete_tree_stats+')')
con.commit()

In [104]:
estimated_high_sampling_tree = estimated_trees[3]
estimated_high_sampling_tree
estimated_high_sampling_tree_stats = os.popen('./find_monophyletic.py -t '+estimated_high_sampling_tree+' -f nexus -l '+target_type).readlines()
estimated_high_sampling_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(estimated_high_sampling_tree_stats) if i[0] in [1, 3, 5, 7]]
estimated_high_sampling_tree_stats = ','.join(estimated_high_sampling_tree_stats)
estimated_high_sampling_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'estimated_high_sampling_tree\','+estimated_high_sampling_tree_stats+')')
con.commit()

In [106]:
estimated_low_sampling_tree = estimated_trees[2]
estimated_low_sampling_tree
estimated_low_sampling_tree_stats = os.popen('./find_monophyletic.py -t '+estimated_low_sampling_tree+' -f nexus -l '+target_type).readlines()
estimated_low_sampling_tree_stats = [re.sub('\n', '', i[1]) for i in enumerate(estimated_low_sampling_tree_stats) if i[0] in [1, 3, 5, 7]]
estimated_low_sampling_tree_stats = ','.join(estimated_low_sampling_tree_stats)
estimated_low_sampling_tree_stats
cur.execute('INSERT INTO '+table_name+' VALUES ('+'\'estimated_low_sampling_tree\','+estimated_low_sampling_tree_stats+')')
con.commit()

True